In [1]:
# Clone your GitHub repo (you’ll be prompted to authorize if it's private)
!git clone https://github.com/colterwood/LHL-final-final-project.git

Cloning into 'LHL-final-final-project'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 83 (delta 39), reused 19 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 1.11 MiB | 1.58 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from io import StringIO

In [7]:
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table", id="wnba_tgl_basic")

In [6]:
url = "https://www.basketball-reference.com/wnba/teams/NYL/2024/gamelog/"

if table is None:
    print("Game log table not found in main HTML.")
else:
    df = pd.read_html(str(table))[0]
    print(df.head())

  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk                 G#               Date Unnamed: 3_level_1   
0                  1                  1         2024-05-14                  @   
1                  2                  2         2024-05-16                  @   
2                  3                  3         2024-05-18                NaN   
3                  4                  4         2024-05-20                NaN   
4                  5                  5         2024-05-23                NaN   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Unnamed: 7_level_0  \
                 Opp                W/L Unnamed: 6_level_1                 Tm   
0                WAS                  W                NaN                 85   
1                IND                  W                NaN                102   
2                IND                  W                NaN                 91   
3                SEA       

<ipython-input-6-f175be36b8a7>:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [ ]:
# Print each column name on its own line
for col in df.columns:
    print(col)

('Unnamed: 0_level_0', 'Rk')
('Unnamed: 1_level_0', 'G#')
('Unnamed: 2_level_0', 'Date')
('Unnamed: 3_level_0', 'Unnamed: 3_level_1')
('Unnamed: 4_level_0', 'Opp')
('Unnamed: 5_level_0', 'W/L')
('Unnamed: 6_level_0', 'Unnamed: 6_level_1')
('Unnamed: 7_level_0', 'Tm')
('Unnamed: 8_level_0', 'Opp')
('Unnamed: 9_level_0', 'Unnamed: 9_level_1')
('Team', 'FG')
('Team', 'FGA')
('Team', 'FG%')
('Team', '3P')
('Team', '3PA')
('Team', '3P%')
('Team', 'FT')
('Team', 'FTA')
('Team', 'FT%')
('Team', 'ORB')
('Team', 'TRB')
('Team', 'AST')
('Team', 'STL')
('Team', 'BLK')
('Team', 'TOV')
('Team', 'PF')
('Unnamed: 26_level_0', 'Unnamed: 26_level_1')
('Opponent', 'FG')
('Opponent', 'FGA')
('Opponent', 'FG%')
('Opponent', '3P')
('Opponent', '3PA')
('Opponent', '3P%')
('Opponent', 'FT')
('Opponent', 'FTA')
('Opponent', 'FT%')
('Opponent', 'ORB')
('Opponent', 'TRB')
('Opponent', 'AST')
('Opponent', 'STL')
('Opponent', 'BLK')
('Opponent', 'TOV')
('Opponent', 'PF')


In [10]:
# Combine both levels of MultiIndex columns as-is, joined by an underscore
df.columns = [f"{a}_{b}" for a, b in df.columns]

In [11]:
# Print each column name on its own line
for col in df.columns:
    print(col)

Unnamed: 0_level_0_Rk
Unnamed: 1_level_0_G#
Unnamed: 2_level_0_Date
Unnamed: 3_level_0_Unnamed: 3_level_1
Unnamed: 4_level_0_Opp
Unnamed: 5_level_0_W/L
Unnamed: 6_level_0_Unnamed: 6_level_1
Unnamed: 7_level_0_Tm
Unnamed: 8_level_0_Opp
Unnamed: 9_level_0_Unnamed: 9_level_1
Team_FG
Team_FGA
Team_FG%
Team_3P
Team_3PA
Team_3P%
Team_FT
Team_FTA
Team_FT%
Team_ORB
Team_TRB
Team_AST
Team_STL
Team_BLK
Team_TOV
Team_PF
Unnamed: 26_level_0_Unnamed: 26_level_1
Opponent_FG
Opponent_FGA
Opponent_FG%
Opponent_3P
Opponent_3PA
Opponent_3P%
Opponent_FT
Opponent_FTA
Opponent_FT%
Opponent_ORB
Opponent_TRB
Opponent_AST
Opponent_STL
Opponent_BLK
Opponent_TOV
Opponent_PF


In [12]:
# Drop junk columns with meaningless names
df = df.drop(columns=[
    "Unnamed: 6_level_0_Unnamed: 6_level_1",
    "Unnamed: 9_level_0_Unnamed: 9_level_1",
    "Unnamed: 26_level_0_Unnamed: 26_level_1"
])

In [13]:
# rename the messy column name to 'home_away'
df = df.rename(columns={'Unnamed: 3_level_0_Unnamed: 3_level_1': 'home_away'})

# convert values: '@' becomes 2 (away), everything else becomes 1 (home)
df['home_away'] = df['home_away'].apply(lambda x: 2 if x == '@' else 1)

In [14]:
# count how many home (1) and away (2) games
df['home_away'].value_counts()

,count
home_away,
1,22
2,20


In [15]:
# Rename key columns to clean names
df = df.rename(columns={
    "Unnamed: 0_level_0_Rk": "Rk",
    "Unnamed: 1_level_0_G#": "G#",
    "Unnamed: 2_level_0_Date": "Date",
    "Unnamed: 4_level_0_Opp": "Opp",
    "Unnamed: 5_level_0_W/L": "W/L",
    "Unnamed: 7_level_0_Tm": "Team_Score",
    "Unnamed: 8_level_0_Opp": "Opp_Score"
})

In [16]:
# Print each column name on its own line
for col in df.columns:
    print(col)

Rk
G#
Date
home_away
Opp
W/L
Team_Score
Opp_Score
Team_FG
Team_FGA
Team_FG%
Team_3P
Team_3PA
Team_3P%
Team_FT
Team_FTA
Team_FT%
Team_ORB
Team_TRB
Team_AST
Team_STL
Team_BLK
Team_TOV
Team_PF
Opponent_FG
Opponent_FGA
Opponent_FG%
Opponent_3P
Opponent_3PA
Opponent_3P%
Opponent_FT
Opponent_FTA
Opponent_FT%
Opponent_ORB
Opponent_TRB
Opponent_AST
Opponent_STL
Opponent_BLK
Opponent_TOV
Opponent_PF


In [8]:
# Extract WNBA team abbreviations from the gamelog dropdown menu
team_abbrevs = []
dropdown = soup.find("div", {"id": "inner_nav"})
if dropdown:
    for a in dropdown.find_all("a", href=True):
        href = a["href"]
        if "/wnba/teams/" in href and href.endswith("/gamelog/"):
            abbrev = href.split("/")[3]
            if abbrev not in team_abbrevs:
                team_abbrevs.append(abbrev)

print(team_abbrevs)

['NYL']


In [9]:
# List of team codes for 2024 season
teams = ['ATL', 'CHI', 'CON', 'DAL', 'IND', 'LAS', 'MIN', 'NYL', 'PHO', 'SEA', 'WAS', 'LVA']

# Base URL pattern
base_url = "https://www.basketball-reference.com/wnba/teams/{team}/2024/gamelog/"

# Empty list to store data from each team
frames = []

for team in teams:
    url = base_url.format(team=team)
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", id="wnba_tgl_basic")

    if table is not None:
        df_team = pd.read_html(StringIO(str(table)))[0]

        # Combine multi-level headers
        df_team.columns = [f"{a}_{b}" for a, b in df_team.columns]

        # Drop extra unnamed columns
        drop_cols = [
            'Unnamed: 6_level_0_Unnamed: 6_level_1',
            'Unnamed: 9_level_0_Unnamed: 9_level_1',
            'Unnamed: 26_level_0_Unnamed: 26_level_1'
        ]
        df_team = df_team.drop(columns=drop_cols, errors='ignore')

        # Rename key columns
        df_team = df_team.rename(columns={
            'Unnamed: 0_level_0_Rk': 'Rk',
            'Unnamed: 1_level_0_G#': 'G#',
            'Unnamed: 2_level_0_Date': 'Date',
            'Unnamed: 4_level_0_Opp': 'Opp',
            'Unnamed: 5_level_0_W/L': 'W/L',
            'Unnamed: 7_level_0_Tm': 'Team_Score',
            'Unnamed: 8_level_0_Opp': 'Opp_Score'
        })

        # Add team identifier as the first column
        df_team.insert(0, "Team", team)

        frames.append(df_team)
    else:
        print(f"No table found for {team}")

# Combine into single DataFrame
df = pd.concat(frames, ignore_index=True)

In [10]:
df.head()

,Team,Rk,G#,Date,Unnamed: 3_level_0_Unnamed: 3_level_1,Opp,W/L,Team_Score,Opp_Score,Team_FG,...,Opponent_FT,Opponent_FTA,Opponent_FT%,Opponent_ORB,Opponent_TRB,Opponent_AST,Opponent_STL,Opponent_BLK,Opponent_TOV,Opponent_PF
0,ATL,1,1,2024-05-15,@,LAS,W,92,81,34,...,13,22,.591,8,35,24,8,2,11,17
1,ATL,2,2,2024-05-18,@,PHO,L,85,88,27,...,28,32,.875,7,38,17,10,5,12,22
2,ATL,3,3,2024-05-21,NaN,DAL,W,83,78,30,...,16,20,.800,11,35,17,6,6,14,20
3,ATL,4,4,2024-05-26,NaN,MIN,L,79,92,31,...,17,23,.739,5,31,23,7,4,10,17
4,ATL,5,5,2024-05-29,@,WAS,W,73,67,26,...,7,7,1.000,9,31,21,6,3,13,17


In [11]:
# count unique values in column
df['Unnamed: 3_level_0_Unnamed: 3_level_1'].value_counts()

,count
Unnamed: 3_level_0_Unnamed: 3_level_1,
@,240


In [12]:
# count null values in the original home/away column
df['Unnamed: 3_level_0_Unnamed: 3_level_1'].isna().sum()

np.int64(264)

In [13]:
# count how many rows each Team has
df['Team'].value_counts()

,count
Team,
ATL,42
CHI,42
CON,42
DAL,42
IND,42
LAS,42
MIN,42
NYL,42
PHO,42


In [14]:
# count how many rows each Opp has
df['Opp'].value_counts()

,count
Opp,
LAS,40
PHO,40
DAL,40
MIN,40
WAS,40
LVA,40
CON,40
NYL,40
CHI,40


In [16]:
# show rows where 'Opp' column contains the string 'Opp'
df[df['Opp'] == 'Opp']

,Team,Rk,G#,Date,Unnamed: 3_level_0_Unnamed: 3_level_1,Opp,W/L,Team_Score,Opp_Score,Team_FG,...,Opponent_FT,Opponent_FTA,Opponent_FT%,Opponent_ORB,Opponent_TRB,Opponent_AST,Opponent_STL,Opponent_BLK,Opponent_TOV,Opponent_PF
21,ATL,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
63,CHI,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
105,CON,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
147,DAL,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
189,IND,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
231,LAS,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
273,MIN,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
315,NYL,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
357,PHO,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
399,SEA,Rk,G#,Date,NaN,Opp,W/L,Tm,Opp,FG,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF


In [18]:
# drop rows where 'Opp' column has the string 'Opp'
df = df[df['Opp'] != 'Opp']

In [19]:
# rename the messy column name to 'home_away'
df = df.rename(columns={'Unnamed: 3_level_0_Unnamed: 3_level_1': 'home_away'})

# convert values: '@' becomes 2 (away), everything else becomes 1 (home)
df['home_away'] = df['home_away'].apply(lambda x: 2 if x == '@' else 1)

In [20]:
# count how many home (1) and away (2) games
df['home_away'].value_counts()

,count
home_away,
1,252
2,240


In [21]:
# count nulls in each column
df.isnull().sum()

,0
Team,0
Rk,12
G#,12
Date,12
home_away,0
Opp,12
W/L,12
Team_Score,12
Opp_Score,12
Team_FG,0


In [23]:
# show rows where 'G#' is null
df[df['G#'].isnull()]

,Team,Rk,G#,Date,home_away,Opp,W/L,Team_Score,Opp_Score,Team_FG,...,Opponent_FT,Opponent_FTA,Opponent_FT%,Opponent_ORB,Opponent_TRB,Opponent_AST,Opponent_STL,Opponent_BLK,Opponent_TOV,Opponent_PF
20,ATL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
62,CHI,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
104,CON,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
146,DAL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
188,IND,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
230,LAS,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
272,MIN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
314,NYL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
356,PHO,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent
398,SEA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,Team,...,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent,Opponent


In [25]:
# drop rows where 'G#' is null
df = df[df['G#'].notnull()]

In [26]:
# count how many home (1) and away (2) games
df['home_away'].value_counts()

,count
home_away,
2,240
1,240


In [27]:
df.head()

,Team,Rk,G#,Date,home_away,Opp,W/L,Team_Score,Opp_Score,Team_FG,...,Opponent_FT,Opponent_FTA,Opponent_FT%,Opponent_ORB,Opponent_TRB,Opponent_AST,Opponent_STL,Opponent_BLK,Opponent_TOV,Opponent_PF
0,ATL,1,1,2024-05-15,2,LAS,W,92,81,34,...,13,22,.591,8,35,24,8,2,11,17
1,ATL,2,2,2024-05-18,2,PHO,L,85,88,27,...,28,32,.875,7,38,17,10,5,12,22
2,ATL,3,3,2024-05-21,1,DAL,W,83,78,30,...,16,20,.800,11,35,17,6,6,14,20
3,ATL,4,4,2024-05-26,1,MIN,L,79,92,31,...,17,23,.739,5,31,23,7,4,10,17
4,ATL,5,5,2024-05-29,2,WAS,W,73,67,26,...,7,7,1.000,9,31,21,6,3,13,17


In [28]:
print(df["Team"].unique())

['ATL' 'CHI' 'CON' 'DAL' 'IND' 'LAS' 'MIN' 'NYL' 'PHO' 'SEA' 'WAS' 'LVA']


In [ ]:
# Save to data folder
df.to_csv(r"C:\Users\colte\LHL-final-project\LHL-final-final-project\data\2024_basketball_reference_gamelog.csv", index=False)

In [5]:
# load the player game logs CSV from the data folder
df = pd.read_csv("LHL-final-final-project/data/2024_basketball_reference_gamelog.csv")

,Team,Rk,G#,Date,Opp,W/L,Team_Score,Opp_Score,Team_FG,Team_FGA,...,Opponent_FT,Opponent_FTA,Opponent_FT%,Opponent_ORB,Opponent_TRB,Opponent_AST,Opponent_STL,Opponent_BLK,Opponent_TOV,Opponent_PF
0,ATL,1,1,2024-05-15,LAS,W,92,81,34,67,...,13,22,.591,8,35,24,8,2,11,17
1,ATL,2,2,2024-05-18,PHO,L,85,88,27,66,...,28,32,.875,7,38,17,10,5,12,22
2,ATL,3,3,2024-05-21,DAL,W,83,78,30,72,...,16,20,.800,11,35,17,6,6,14,20
3,ATL,4,4,2024-05-26,MIN,L,79,92,31,68,...,17,23,.739,5,31,23,7,4,10,17
4,ATL,5,5,2024-05-29,WAS,W,73,67,26,62,...,7,7,1.000,9,31,21,6,3,13,17


In [ ]:
# preview
df.head()

In [29]:
# make all column headers lowercase
df.columns = df.columns.str.lower()

In [30]:
# print each column with its dtype in a readable list
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- rk: object
- g#: object
- date: object
- home_away: int64
- opp: object
- w/l: object
- team_score: object
- opp_score: object
- team_fg: object
- team_fga: object
- team_fg%: object
- team_3p: object
- team_3pa: object
- team_3p%: object
- team_ft: object
- team_fta: object
- team_ft%: object
- team_orb: object
- team_trb: object
- team_ast: object
- team_stl: object
- team_blk: object
- team_tov: object
- team_pf: object
- opponent_fg: object
- opponent_fga: object
- opponent_fg%: object
- opponent_3p: object
- opponent_3pa: object
- opponent_3p%: object
- opponent_ft: object
- opponent_fta: object
- opponent_ft%: object
- opponent_orb: object
- opponent_trb: object
- opponent_ast: object
- opponent_stl: object
- opponent_blk: object
- opponent_tov: object
- opponent_pf: object


In [31]:
# replace '%' with '_pct' in column names
df.columns = df.columns.str.replace('%', '_pct', regex=False)

In [32]:
# rename w/l to win_loss
df = df.rename(columns={'w/l': 'win_loss'})

# rename g# to g_num
df = df.rename(columns={'g#': 'g_num'})

In [33]:
# convert values: W becomes 1 (win), everything else becomes 2 (loss)
df['win_loss'] = df['win_loss'].apply(lambda x: 1 if x == 'W' else 2)

In [34]:
# count how many wins (1) and losses (2)
df['win_loss'].value_counts()

,count
win_loss,
1,240
2,240


In [35]:
# print each column with its dtype in a readable list
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- rk: object
- g_num: object
- date: object
- home_away: int64
- opp: object
- win_loss: int64
- team_score: object
- opp_score: object
- team_fg: object
- team_fga: object
- team_fg_pct: object
- team_3p: object
- team_3pa: object
- team_3p_pct: object
- team_ft: object
- team_fta: object
- team_ft_pct: object
- team_orb: object
- team_trb: object
- team_ast: object
- team_stl: object
- team_blk: object
- team_tov: object
- team_pf: object
- opponent_fg: object
- opponent_fga: object
- opponent_fg_pct: object
- opponent_3p: object
- opponent_3pa: object
- opponent_3p_pct: object
- opponent_ft: object
- opponent_fta: object
- opponent_ft_pct: object
- opponent_orb: object
- opponent_trb: object
- opponent_ast: object
- opponent_stl: object
- opponent_blk: object
- opponent_tov: object
- opponent_pf: object


In [36]:
# count nulls in each column
df.isnull().sum()

,0
team,0
rk,0
g_num,0
date,0
home_away,0
opp,0
win_loss,0
team_score,0
opp_score,0
team_fg,0


In [37]:
# convert to datetime (if not already)
df['date'] = pd.to_datetime(df['date'])

# split into year, month, day
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [38]:
# drop 'date' column
df = df.drop(columns=['date'])

# build new column order without year/month/day
cols = [col for col in df.columns if col not in ['year', 'month', 'day']]

# find where to insert
insert_at = cols.index('home_away')

# insert year/month/day before 'age'
new_order = cols[:insert_at] + ['year', 'month', 'day'] + cols[insert_at:]

# reorder
df = df[new_order]

In [39]:
df.head()

,team,rk,g_num,year,month,day,home_away,opp,win_loss,team_score,...,opponent_ft,opponent_fta,opponent_ft_pct,opponent_orb,opponent_trb,opponent_ast,opponent_stl,opponent_blk,opponent_tov,opponent_pf
0,ATL,1,1,2024,5,15,2,LAS,1,92,...,13,22,.591,8,35,24,8,2,11,17
1,ATL,2,2,2024,5,18,2,PHO,2,85,...,28,32,.875,7,38,17,10,5,12,22
2,ATL,3,3,2024,5,21,1,DAL,1,83,...,16,20,.800,11,35,17,6,6,14,20
3,ATL,4,4,2024,5,26,1,MIN,2,79,...,17,23,.739,5,31,23,7,4,10,17
4,ATL,5,5,2024,5,29,2,WAS,1,73,...,7,7,1.000,9,31,21,6,3,13,17


In [40]:
# drop the 'rk' column
df = df.drop(columns=['rk'])

In [42]:
# print each column with its dtype in a readable list
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- g_num: object
- year: int32
- month: int32
- day: int32
- home_away: int64
- opp: object
- win_loss: int64
- team_score: object
- opp_score: object
- team_fg: object
- team_fga: object
- team_fg_pct: object
- team_3p: object
- team_3pa: object
- team_3p_pct: object
- team_ft: object
- team_fta: object
- team_ft_pct: object
- team_orb: object
- team_trb: object
- team_ast: object
- team_stl: object
- team_blk: object
- team_tov: object
- team_pf: object
- opponent_fg: object
- opponent_fga: object
- opponent_fg_pct: object
- opponent_3p: object
- opponent_3pa: object
- opponent_3p_pct: object
- opponent_ft: object
- opponent_fta: object
- opponent_ft_pct: object
- opponent_orb: object
- opponent_trb: object
- opponent_ast: object
- opponent_stl: object
- opponent_blk: object
- opponent_tov: object
- opponent_pf: object


In [43]:
df.to_csv("2024_basketball_reference_gamelog.csv", index=False)

from google.colab import files

files.download("2024_basketball_reference_gamelog.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
# list of columns that should be numeric
cols_to_convert = [col for col in df.columns if col not in ['team', 'opp']]

# convert all non-string columns to numeric, coercing errors
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [45]:
# show columns with > 0 null values
df.isnull().sum()[df.isnull().sum() > 0]

,0


In [46]:
# print each column with its dtype in a readable list
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- g_num: int64
- year: int32
- month: int32
- day: int32
- home_away: int64
- opp: object
- win_loss: int64
- team_score: int64
- opp_score: int64
- team_fg: int64
- team_fga: int64
- team_fg_pct: float64
- team_3p: int64
- team_3pa: int64
- team_3p_pct: float64
- team_ft: int64
- team_fta: int64
- team_ft_pct: float64
- team_orb: int64
- team_trb: int64
- team_ast: int64
- team_stl: int64
- team_blk: int64
- team_tov: int64
- team_pf: int64
- opponent_fg: int64
- opponent_fga: int64
- opponent_fg_pct: float64
- opponent_3p: int64
- opponent_3pa: int64
- opponent_3p_pct: float64
- opponent_ft: int64
- opponent_fta: int64
- opponent_ft_pct: float64
- opponent_orb: int64
- opponent_trb: int64
- opponent_ast: int64
- opponent_stl: int64
- opponent_blk: int64
- opponent_tov: int64
- opponent_pf: int64


In [47]:
df.to_csv("2024_basketball_reference_gamelog.csv", index=False)

from google.colab import files

files.download("2024_basketball_reference_gamelog.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>